# Eaglesong

## Introduction

In this notebook, we will cover the essentials of building a telegram bots with `eaglesong` library. Telegram, as well as other messengers, offer the "per request" approach: the chatbot is given the incoming message and must produce the result. However, in case of longer conversations, it is easier to code the process from the chatbot's point of view: say this, listen to human, parse the input, say something else. `eaglesong` provides exactly this possibility. You write _chat flows_ like this:

```
yield "First question"
first_answer = yield Listen()

yield "Second question"
second_answer = yield Listen()
```

and then control system builds a Telegram bot (or bot for other media) from such flows.

There are several onboarding scripts in this directory, each of them representing one chatbot. The code and comments for these chatbots are placed in this notebook for convenience. Aside from reading, you can:

1. Run the scripts as a Telegram bot. 
2. `eaglesong` provides a testbed environment to run chatbots in the notebook, see `Sandbox` notebook. This environment is not as well-studied as Telegram, so problems may appear when executing bots.
3. Read and run the tests. All the bots in the demos are tested with unit tests, and you can find them in `/kaia_tests/test_eaglesong/test_demo_skills`.

### Advanced notes

When a Telegram bot receives the very request for an update, it creates an iterator over
`main` function, and pulls commands from it until it's `Listen`. At this point the request
is considered complete, iterator is stored and the bot return the control to Telegram loop.
On the second request, it will restore the iterator and continue with the updated `context`
field from the exactly same point where it was interrupted.

Some people suggested `yield` approach can be replaced with async/await,
keeping the logic of the conversation flow intact.
Some other people, however, offered arguments why these approaches,
although similar and based on the same design pattern, are not equivalent in Python
and hence async/await cannot be used in this particular case.

* Unfortunately, my understanding of async/await does not allow me to answer this question with certainty.
    If someone wants to reimplement eaglesong with async/await, this and further demos provide
    a good understanding of the use cases that need to be considered.
* In general, I don't believe
    writing `await` instead of `yield` will improve anything. Although, we could benefit
    from some standard await management from `asyncio`.
* Both approaches should be able to coexist side-by-side with `Automaton` class abstraction.

## Setting things up

First we run the `eaglesong` unittest. If the following cell doesn't print `Success`, something if wrong and you need to fix it before proceeding.

In [1]:
import subprocess
import sys
from IPython.display import clear_output


assert 0 == subprocess.call([sys.executable, '-m', 'unittest', 'discover', '../../kaia_tests/test_eaglesong'])
clear_output()
print('Success!')

Success!


Before running the bots from `demos/eaglesong`, you will need:

1. Contact `@BotFather` bot on Telegram and register your chatbot. As the result, you will obtain an API key that looks like this: `0000000000:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`
2. Create an `environment.env` file in the repository's root. This file is already in `.gitignore`, so don't be afraid to accidentaly push its contents.
3. Place `KAIA_TEST_BOT=<YOUR_API_KEY>` in the `environment.env` file.

The following cell checks if this was successful:

In [2]:
from kaia.infra import Loc
import os


assert 'KAIA_TEST_BOT' in os.environ, 'Bot API KEY was not found'
print('Success!')

Success!


`Loc` is a class in `kaia.infra` that manages the environment. In particular, it reads the environment variables from ENV file, so you don't need to do it manually in any way.

## Bots' code

These bots will help you to understand how to design the chat-flow with eaglesong, and are sufficient to write a easy-to-medium chatbot yourself.

In [3]:
from kaia.infra.demos import py_to_notebook
from yo_fluq_ds import *

py_to_notebook(Query.folder('.').where(lambda z: z.name<='example_10' and 'example_01'<=z.name).order_by(lambda z: z.name).to_list())